In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv('review.csv')

In [3]:
meta = pd.read_csv('meta.csv')

col = list(df.columns)
col.append('title')

df = df.merge(meta, on=['asin'])[col]

df.head()

C:\Users\Elga Ridho Maulana\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3057: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,asin,helpful,overall,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime,title
0,616719923X,"[0, 0]",4.0,Just another flavor of Kit Kat but the taste i...,"06 1, 2013",A1VEELTKS8NLZB,Amazon Customer,Good Taste,1370044800,Japanese Kit Kat Maccha Green Tea Flavor (5 Ba...
1,616719923X,"[0, 1]",3.0,I bought this on impulse and it comes from Jap...,"05 19, 2014",A14R9XMZVJ6INB,amf0001,"3.5 stars, sadly not as wonderful as I had hoped",1400457600,Japanese Kit Kat Maccha Green Tea Flavor (5 Ba...
2,616719923X,"[3, 4]",4.0,Really good. Great gift for any fan of green t...,"10 8, 2013",A27IQHDZFQFNGG,Caitlin,Yum!,1381190400,Japanese Kit Kat Maccha Green Tea Flavor (5 Ba...
3,616719923X,"[0, 0]",5.0,"I had never had it before, was curious to see ...","05 20, 2013",A31QY5TASILE89,DebraDownSth,Unexpected flavor meld,1369008000,Japanese Kit Kat Maccha Green Tea Flavor (5 Ba...
4,616719923X,"[1, 2]",4.0,I've been looking forward to trying these afte...,"05 26, 2013",A2LWK003FFMCI5,Diana X.,"Not a very strong tea flavor, but still yummy ...",1369526400,Japanese Kit Kat Maccha Green Tea Flavor (5 Ba...


## Demographical Filtering

Kita akan melakukan demographic filtering dengan melakukan perhitungan dari rating dan jumlah review dari suatu produk. Kita tidak dapat melihat produk dengan rating paling tinggi menjadi produk rekomendasi, karena belum tentu rating yang tinggi memiliki jumlah review yang banyak. Rata-rata rating 5 tetapi hanya memiliki 10 review tentu tidak lebih baik daripada rata-rata rating 4.8 tetapi memiliki 1000 rating.

In [4]:
df_demographical = df.groupby('title').mean()[['overall']].reset_index()

In [5]:
df_demographical['count_review'] = df.groupby('title').count().reset_index()['overall']

In [6]:
# fungsi yang melakukan perhitungan 
def weighted_rating(row):
    v = row['count_review']
    m = 60
    r = row['overall']
    c = df_demographical['overall'].mean()
    wr = ((v / (v+m)) * r) + ((m/(v+m))* c)
    return wr

df_demographical['score'] = df_demographical.apply(weighted_rating, axis=1)

In [7]:
df_demographical.head()

,title,overall,count_review,score
0,&quot;Raspberry Leaf C/S Organic 1 pound,4.666667,6,4.380590
1,1 Gallon 100% Organic Extra Virgin Raw Coconut...,4.833333,6,4.395742
2,1 Gallon Snappy Butter Burst Oil,4.266667,15,4.334919
3,100 Blooms of Peruvian Lilies,3.888889,9,4.291579
4,100 Calorie Packs Lorna Doone Shortbread Cooki...,4.857143,7,4.404760


In [8]:
df_demographical = df_demographical.merge(meta, on='title')[['title', 'overall', 'count_review', 'score', 'description']]

In [9]:
df_demographical.sort_values(by=['score'], ascending=False).head(20)

,title,overall,count_review,score,description
8298,"Viva Labs Organic Extra Virgin Coconut Oil, 16...",4.790698,258,4.707921,The #1 Coconut Oil for Freshness and PurityEve...
5521,"Nature's Way Coconut Oil, 16 Ounce",4.775330,227,4.686826,Organic Coconut Oil by Nature's Way 16 oz Liqu...
5794,"Nutiva Organic Extra Virgin Coconut Oil, 15-Ou...",4.773756,221,4.683697,"Nutiva's cold-pressed, Organic extra-virgin co..."
6118,"Pamela's Ultimate Baking and Pancake Mix, 4-Po...",4.793103,145,4.663995,"Create light and fluffy pancakes and waffles, ..."
5805,"Nutiva Organic Virgin Coconut Oil, 54-Ounce Co...",4.862069,87,4.653870,NaN
8596,Y.S. Eco Bee Farms Raw Honey - 22 oz,4.819048,105,4.649206,Health Honey - Raw Unheated Unprocessed by Y.S...
7835,The Spice Lab's Pure Himalayan Natural Unproce...,4.842697,89,4.645094,The Spice Lab Finely Ground Pure Himalayan Salt
5784,"Nutiva Organic Chia Seeds, 12-Ounce Bag",4.779528,127,4.642347,"Chia seeds are a superfood packed with fiber, ..."
8158,U.S.A Grown Garbanzo Beans 5 LBS | 100% Non-Ir...,4.945455,55,4.635817,Buy Garbanzo Beans grown from the fertile soil...
3725,"Huy Fong, Sriracha Hot Chili Sauce, 17 Ounce B...",4.863014,73,4.632473,Sriracha is made from sun ripen chilies which ...


Kita berhasil membuat sistem rekomendasi dengan menggunakan demographic filtering. Produk dengan rating dan jumlah review yang paling baik adalah 'Viva Las Organic Coconut Oil'. 3 Produk utama adalah coconut oil, selanjutnya Pancake. Terdapat juga Madu di posisi ke-6.

## Content Based

Selain demographic filtering, sistem rekomendasi juga dapat dibuat berdasarkan Content Based. Jadi, sistem akan merekomendasikan produk yang memiliki kemiripan suatu hal. Dalam projek ini, sistem akan merekomendasikan produk yang memiliki kemiripin dalam title (judul) dan deskripsi produk.

**based by title**

In [10]:
content_based_title = df_demographical[['title', 'description']]

In [11]:
from sklearn.feature_extraction.text import CountVectorizer

In [12]:
count_vec = CountVectorizer()

In [13]:
count_vec_title = count_vec.fit_transform(content_based_title['title'])

In [14]:
# make count_vec_new_genre to dataframe : cv_df
cv_df = pd.DataFrame(count_vec_title.todense(), columns=count_vec.get_feature_names(),
            index=content_based_title['title'])

# look the head of the dataframe
cv_df.head()

,00,000,002,00649056,01,01170,01317,02,02oz,03,...,zipfizz,zoe,zola,zombie,zoo,zotz,zsweet,zulu,zuppa,zveet
title,,,,,,,,,,,,,,,,,,,,,
&quot;Raspberry Leaf C/S Organic 1 pound,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1 Gallon 100% Organic Extra Virgin Raw Coconut Oil,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1 Gallon Snappy Butter Burst Oil,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
100 Blooms of Peruvian Lilies,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
100 Blooms of Peruvian Lilies,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
from sklearn.metrics.pairwise import cosine_similarity

cos_sim = cosine_similarity(count_vec_title)

cos_sim_title = pd.DataFrame(cos_sim)

In [16]:
index_to_search = df_demographical[df_demographical['title'] == "Y.S. Eco Bee Farms Raw Honey - 22 oz"].index[0]
similar_index = pd.Series(cos_sim_title[index_to_search]).sort_values(ascending=False).head(15).index

df_demographical.loc[similar_index]

,title,overall,count_review,score,description
8596,Y.S. Eco Bee Farms Raw Honey - 22 oz,4.819048,105,4.649206,Health Honey - Raw Unheated Unprocessed by Y.S...
6546,Raw Tupelo Honey YS Eco Bee Farms 13.5 oz Liquid,5.000000,10,4.444556,Raw Tupelo Honey by YS Eco Bee Farms 13.5 oz L...
8598,"YS Eco Bee Farms RAW HONEY - Raw, Unfiltered, ...",4.666667,12,4.404430,"Naturally Crystallized, Energy Packed, True Ra..."
8599,YS Organic Bee Farms - Healthy Honey (Raw) - 1...,4.538462,13,4.385191,"Fully crystallized, energy packed, true raw so..."
6526,Raw Buckwheat Honey YS Organic Bee Farms 13.5 ...,4.428571,7,4.359984,Raw Buckwheat Honey by YS Organic Bee Farms 13...
8602,YS Organic Bee Farms Raw Honey -- 2 lbs,4.846154,13,4.439985,As the world pioneer in certified organic bee ...
8600,"YS Organic Bee Farms - Organic Honey, 16 oz gel",4.285714,7,4.345059,Certified Organic Raw Honey by YS Organic Bee ...
3618,"Honey Granules, 8 oz.",4.222222,9,4.335057,"Honey Granules, 8 oz."
8393,Wee Bee Naturally Raw Honey -- 1 lb Each,4.230769,13,4.330396,It is a well known fact that honey is an extre...
8601,YS Organic Bee Farms CERTIFIED ORGANIC RAW HON...,4.760870,92,4.599467,YS Organic - Certified Organic Honey is harves...


In [17]:
index_to_search = df_demographical[df_demographical['title'] == "Nutiva Organic Chia Seeds, 12-Ounce Bag"].index[0]
similar_index = pd.Series(cos_sim_title[index_to_search]).sort_values(ascending=False).head(15).index

df_demographical.loc[similar_index]

,title,overall,count_review,score,description
5784,"Nutiva Organic Chia Seeds, 12-Ounce Bag",4.779528,127,4.642347,"Chia seeds are a superfood packed with fiber, ..."
5788,"Nutiva Organic Chia Seeds, White, 12-Ounce",4.615385,13,4.398890,"Chia seeds are a superfood packed with fiber, ..."
5787,"Nutiva Organic Chia Seeds, Milled, 12-Ounce",4.311111,90,4.327460,"Chia seeds are a superfood packed with fiber, ..."
5785,"Nutiva Organic Chia Seeds, 12-Ounce Bags (Pack...",4.937500,16,4.475249,"Chia seeds are a superfood packed with fiber, ..."
5786,"Nutiva Organic Chia Seeds, 3-Pound Bags",4.764706,17,4.443103,Benfits of Chia Seeds: 3x the iron of spinach...
5808,"Nutiva Organic White Chia Seeds, 3-pound",4.285714,7,4.345059,Benfits of Chia Seeds: 3x the iron of spinach...
548,"Barlean's Organic Oils Organic Chia Seed, 12 O...",4.400000,10,4.358842,Dated back to aztex warriors and used today by...
7250,"Spectrum Essentials Chia Seeds, 12 Ounce Pouch",4.454545,11,4.367872,"At Spectrum, we work with talented, dedicated ..."
5955,"Organic Valley Organic Nonfat Dry Milk Powder,...",4.333333,6,4.350287,Organic Valley Organic Nonfat Dry Milk Powder
8299,"Viva Labs Organic Raw Chia Seeds, 1 lb Bag",5.000000,19,4.507835,The Mighty Chia SeedChia seeds were first harv...


**based by description**

In [18]:
count_vec2 = CountVectorizer()

In [19]:
content_based_title.isnull().sum()

title             0
description    1200
dtype: int64

In [20]:
content_based_description = df_demographical.dropna().reset_index(drop=True)

In [21]:
count_vec_description = count_vec2.fit_transform(content_based_description['description'])

In [22]:
cos_sim_desc = cosine_similarity(count_vec_description)

In [23]:
index_to_search = content_based_description[content_based_description['title'] == "Y.S. Eco Bee Farms Raw Honey - 22 oz"].index[0]
similar_index = pd.Series(cos_sim_desc[index_to_search]).sort_values(ascending=False).head(15).index

content_based_description.loc[similar_index]

,title,overall,count_review,score,description
7420,Y.S. Eco Bee Farms Raw Honey - 22 oz,4.819048,105,4.649206,Health Honey - Raw Unheated Unprocessed by Y.S...
1940,Dutch Gold Honey 100% Organic Wildflower Honey...,4.285714,7,4.345059,The process of creating honey is a tricky one!...
3200,"Horizon Organic 1 % Low Fat Milk, 8-Ounce Asep...",4.150000,20,4.301487,Horizon Organic is committed to the long-term ...
3202,"Horizon Organic Low Fat Milk, Strawberry, 8-Ou...",4.181818,11,4.325619,Horizon Organic is committed to the long-term ...
3201,"Horizon Organic Low Fat Milk, Chocolate, 8-Oun...",4.176471,17,4.313233,Horizon Organic is committed to the long-term ...
3203,"Horizon Organic Low Fat Milk, Vanilla, 8-Ounce...",4.500000,12,4.376652,Horizon Organic is committed to the long-term ...
1316,Certified Organic Hard Red Wheat - 5 Lbs - For...,4.461538,13,4.371492,All Wheat is Not Equal We have had the experi...
4520,"Mount Hagen Organic Instant Regular Coffee, 25...",4.333333,12,4.348874,Mount Hagen instant organic coffee is organica...
4516,Mount Hagen Organic Freeze Dried Instant Coffe...,4.600000,5,4.371061,Mount Hagen instant organic coffee is organica...
6166,Snyder's of Hanover Organic Honey Whole Wheat ...,3.600000,5,4.294138,"A Distinct Line of Organic Pretzels, baked onl..."


**based by title and description**

In [24]:
count_vec = CountVectorizer()

In [25]:
count_vec_title2 = count_vec.fit_transform(content_based_description['title'])

In [26]:
cos_sim_title = cosine_similarity(count_vec_title2)

In [27]:
cos_sim_desc_title = (0.7 * cos_sim_title) + (0.3 * cos_sim_desc)

In [28]:
index_to_search = content_based_description[content_based_description['title'] == "Nature's Way Coconut Oil, 16 Ounce"].index[0]
similar_index = pd.Series(cos_sim_desc_title[index_to_search]).sort_values(ascending=False).head(15).index

content_based_description.loc[similar_index]

,title,overall,count_review,score,description
4727,"Nature's Way Coconut Oil, 16 Ounce",4.775330,227,4.686826,Organic Coconut Oil by Nature's Way 16 oz Liqu...
4672,"Nature'S Way, Coconut Oil Liquid, 10 Oz",5.000000,8,4.428220,Liquid Coconut Premium Oil by Nature's Way 10 ...
7155,"Viva Labs Organic Extra Virgin Coconut Oil, 16...",4.790698,258,4.707921,The #1 Coconut Oil for Freshness and PurityEve...
1442,Coconut Oil - 1 Quart,4.375000,8,4.354690,Finest Quality Coconut Oil - 76 F - 1 Quart sh...
3382,Jarrow Formulas Coconut Oil 16 oz,4.456140,114,4.420224,Coconut Oil 16 oz\n\nDietary Supplement\n\n&#x...
5094,Organic Coconut Oil 14 oz.,4.428571,7,4.359984,Use Spectrum Naturals Organic Coconut Oil for ...
610,Betterbody Foods &amp; Nutrition Organic Cocon...,4.400000,5,4.355676,"Our organic coconut oil, naturally refined fro..."
4966,"Nutiva Organic Chips, Coconut, 1 Pound, 16 Ounce",4.592105,76,4.486169,Nutiva&#x2019;s organic coconut chips come fro...
3818,"La Tourangelle Avocado Oil, 16.9 Ounce Tin",4.631579,19,4.419227,"La Tourangelle Avocado Oil is 100% pure, all-n..."
5287,"Parachute Coconut Oil, 33.8-Ounce",4.500000,6,4.365439,This contain edible oil made from finest cocon...


## Collaborative Filtering

Dalam collaborative filtering, user harus melakukan rating terlebih dahulu terhadap suatu produk agar dapat merasakan rekomendasi ini. Collaborative filtering terdiri dari dua yaitu item based dan user based. Untuk item based, akan dilihat kesamaan suatu produk berdasarkan rating yang diberikan oleh reviewer. Sementara itu, untuk user based akan dilihat kesamaan dari user berdasarkan produk-produk yang dia berikan rating dan review.

**item based collaborative filtering**

In [29]:
item_based= df[['reviewerID', 'title', 'overall']]

In [30]:
item_based.head()

,reviewerID,title,overall
0,A1VEELTKS8NLZB,Japanese Kit Kat Maccha Green Tea Flavor (5 Ba...,4.0
1,A14R9XMZVJ6INB,Japanese Kit Kat Maccha Green Tea Flavor (5 Ba...,3.0
2,A27IQHDZFQFNGG,Japanese Kit Kat Maccha Green Tea Flavor (5 Ba...,4.0
3,A31QY5TASILE89,Japanese Kit Kat Maccha Green Tea Flavor (5 Ba...,5.0
4,A2LWK003FFMCI5,Japanese Kit Kat Maccha Green Tea Flavor (5 Ba...,4.0


In [31]:
# Make pivot table : user_movie_ratings_pivot
item_based_pivot = item_based.pivot_table(index='title', columns='reviewerID', values='overall')

In [32]:
item_based_pivot.shape

(8692, 14681)

In [33]:
item_based_pivot.dropna(thresh=10, inplace=True)

In [34]:
# Fill na with 0 values
item_based_pivot.fillna(0, inplace=True)

In [35]:
item_based_pivot.head()

reviewerID,A00177463W0XWB16A9O05,A022899328A0QROR32DCT,A04309042SDSL8YX2HRR7,A068255029AHTHDXZURNU,A06944662TFWOKKV4GJKX,A1004703RC79J9,A1006HCQDMYC5W,A1008DPSP6KC9J,A100DXY4SLAMPM,A100I4UAHGQCF6,...,AZV3KWJRFXLAX,AZVJHW8TARWV9,AZWIAYHWL2FWE,AZWIEXG6P4Y9W,AZWP97BZPJI1D,AZWRZZAMX90VT,AZXKAH2DE6C8A,AZXON596A1VXC,AZYXC63SS008M,AZZ5ASC403N74
title,,,,,,,,,,,,,,,,,,,,,
1 Gallon Snappy Butter Burst Oil,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"100 Calorie Packs Oreo Thin Crisps, 6-Count Packs (Pack of 2)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
100% Bovine GELATIN 14oz.,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
100% Coconut Milk - 8.5 oz packages (6-pack),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
100% NATURAL 4.4 oz GINGER CANDY,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [36]:
similarity_item = cosine_similarity(item_based_pivot)

df_similarity_item = pd.DataFrame(similarity_item, index=item_based_pivot.index, columns=item_based_pivot.index)

In [37]:
ser1 = (df_similarity_item['Caribou Coffee Caribou Blend, K-Cups for Keurig Brewers, 24-Count'] * (4 - 2.5)).sort_values(ascending=False)
ser2 = (df_similarity_item['The Organic Coffee Co. Java Love, 12 OneCup Single Serve Cups'] * (5 - 2.5)).sort_values(ascending=False)
ser3 = (df_similarity_item['Higgins &amp; Burke Tea Capsules, English Breakfast Package compatible with Keurig K-Cup Brewers, 48 Count'] * (3 - 2.5)).sort_values(ascending=False)

(ser1 + ser2 + ser3).sort_values(ascending=False).head(10)

title
The Organic Coffee Co. Java Love, 12 OneCup Single Serve Cups                                                                 2.582591
Caribou Coffee Caribou Blend, K-Cups for Keurig Brewers, 24-Count                                                             1.639643
Higgins &amp; Burke Tea Capsules, English Breakfast Package compatible with Keurig K-Cup Brewers, 48 Count                    0.572708
Cameron's Donut Shop Single Serve Coffees,  12-Count                                                                          0.413084
Brooklyn Beans Breakfast Blend Decaffeinated CoffeeSingle-cup coffee for Keurig K-Cup Brewers for Keurig Brewers, 40 Count    0.386287
Caribou Coffee Daybreak Morning Blend, K-Cups for Keurig Brewers, 96-Count                                                    0.376626
Tully's Coffee Hawaiian Blend K-Cups                                                                                          0.283739
Green Mountain Coffee, Vermont Country Blend, K-C

In [38]:
ser1 = (df_similarity_item['Pacific Natural Foods Organic Cream Of Mushroom Condensed Soup, 12-Ounce Boxes (Pack of 12)'] * (5 - 2.5)).sort_values(ascending=False)
ser2 = (df_similarity_item['Enfamil Gentlease Infant Formula Milk-Based Powder with Iron, Reusable Tub, 21.5 Ounce (Packaging May Vary)'] * (1 - 2.5)).sort_values(ascending=False)
ser3 = (df_similarity_item['GoPicnic Ready-to-Eat Meals Turkey Pepperoni &amp; Cheese (Pack of 6)'] * (2 - 2.5)).sort_values(ascending=False)
ser4 = (df_similarity_item['Chebe Bread Original Cheese Bread Mix, Gluten Free, 7.5-Ounce Bags (Pack of 8)'] * (1 - 2.5)).sort_values(ascending=False)
ser5 = (df_similarity_item['Kinnikinnick, Smoreable Graham Style Crackers, Wheat Free, Gluten Free, 8-Ounce (Pack of 6)'] * (4 - 2.5)).sort_values(ascending=False)

(ser1 + ser2 + ser3 + ser4 + ser5).sort_values(ascending=False).head(10)

title
Pacific Natural Foods Organic Cream Of Mushroom Condensed Soup, 12-Ounce Boxes (Pack of 12)            2.525519
Kinnikinnick, Smoreable Graham Style Crackers, Wheat Free, Gluten Free, 8-Ounce (Pack of 6)            1.551719
Pacific Natural Foods Organic Cream Of Chicken Condensed Soup, 12-Ounce Boxes (Pack of 12)             0.753928
Kinnikinnick Foods - S'moreables Graham Style Crackers - 8 oz.                                         0.336597
Annie's Homegrown Gluten-Free Rice Pasta &amp; Cheddar Mac &amp; Cheese, 6-Ounce Boxes (Pack of 12)    0.305452
KinnitToos Vanilla Sandwich Creme, 8-Ounce Packages (Pack of 6)                                        0.286758
Glutino Gluten Free Pretzels 14.1oz Bag                                                                0.281718
Glutenfreeda's Instant Oatmeal, Variety Pack, 6-Count Packets (Pack of 8)                              0.275731
Pacific Natural Foods Organic Chicken With Wild Rice Soup, 17.6-Ounce Boxes (Pack of 12)          

In [39]:
df.sort_values('reviewerID')

,asin,helpful,overall,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime,title
140642,B00BFM6OAW,"[0, 0]",3.0,"Its ok, not much of a choc kinda freak. Not my...","04 1, 2013",A00177463W0XWB16A9O05,acethisplace,Hot a choc freak,1364774400,FREE SAMPLE - Kind Peanut Butter Dark Chocolat...
94147,B00474OR8G,"[0, 0]",4.0,Much better than the Daybreak blend. Much Bol...,"04 1, 2013",A00177463W0XWB16A9O05,acethisplace,Like it,1364774400,"Caribou Coffee Caribou Blend, K-Cups for Keuri..."
68143,B0029XDZIK,"[0, 0]",5.0,It is a good stand by coffee you can count on....,"01 6, 2013",A00177463W0XWB16A9O05,acethisplace,Pleasing,1357430400,"Keurig, The Original Donut Shop, 50 Count K-Cu..."
132799,B008TKL6GA,"[2, 2]",5.0,Love the packaging of this coffee. Seems to br...,"02 27, 2013",A00177463W0XWB16A9O05,acethisplace,Awesome,1361923200,"The Organic Coffee Co. Java Love, 12 OneCup Si..."
88359,B003YUW7EK,"[0, 0]",4.0,"I am pleased with the brew, but not enthused. ...","04 1, 2013",A00177463W0XWB16A9O05,acethisplace,It gets ya through,1364774400,"Caribou Coffee Daybreak Morning Blend, K-Cups ..."
127680,B007OSBFY6,"[0, 0]",4.0,I really went through this off brand coffee. ...,"01 6, 2013",A00177463W0XWB16A9O05,acethisplace,Very good for price,1357430400,"Brown Gold 100% Colombian Coffee Capsules, 48-..."
80839,B003C4YIFE,"[1, 1]",5.0,I like the &#34;wake up&#34; factor of this bl...,"02 19, 2013",A00177463W0XWB16A9O05,acethisplace,Heavy coffee,1361232000,"Green Mountain Coffee, Vermont Country Blend, ..."
131665,B008I1XLDQ,"[0, 0]",5.0,I really like this coffee. It has a very uniq...,"01 6, 2013",A00177463W0XWB16A9O05,acethisplace,great sample pack,1357430400,Brooklyn Beans Breakfast Blend Decaffeinated C...
128549,B007TGDXMK,"[0, 0]",5.0,Impressed with the price and taste of this cof...,"04 1, 2013",A00177463W0XWB16A9O05,acethisplace,Yummy,1364774400,San Francisco Bay Coffee Organic Rainforest Bl...
127395,B007OSBE1U,"[0, 1]",5.0,ypiuy y yu 8t 8t o t875t9t uf5o g r6 t9 t7r r...,"01 6, 2013",A00177463W0XWB16A9O05,acethisplace,yuiug7g,1357430400,"Martinson Coffee Capsules, Dark Roast Package ..."


In [40]:
meta[meta['asin']=='B003TO9RSU']['title'].iloc[0]

'Kinnikinnick, Smoreable Graham Style Crackers, Wheat Free, Gluten Free, 8-Ounce (Pack of 6)'

**user based collaborative filtering**

In [41]:
user_based = df[['reviewerID', 'title', 'overall']]

In [42]:
user_based_pivot = user_based.pivot_table(index='reviewerID', columns='title', values='overall')

In [43]:
user_based_pivot.dropna(thresh=10, inplace=True)

In [44]:
user_based_pivot.fillna(0, inplace=True)

In [45]:
similarity_user = cosine_similarity(user_based_pivot)

In [46]:
df_similarity_user = pd.DataFrame(similarity_user, index=user_based_pivot.index, columns=user_based_pivot.index)

In [47]:
df_similarity_user.head()

reviewerID,A00177463W0XWB16A9O05,A022899328A0QROR32DCT,A100VQNP6I54HS,A100WO06OQR8BQ,A100ZQDV7L8PVV,A101RRYMZM4KYV,A1047EDJ84IMAS,A1051DBTLWP5A2,A105S56ODHGJEK,A106Q1A6LGU4DP,...,AZMY6E8B52L2T,AZNUHQSHZHSUE,AZOCADWL9N2H2,AZQA8ZIGS01FG,AZU8GQQW6HESP,AZUUU81LB0NYV,AZV26LP92E6WU,AZWP97BZPJI1D,AZWRZZAMX90VT,AZXKAH2DE6C8A
reviewerID,,,,,,,,,,,,,,,,,,,,,
A00177463W0XWB16A9O05,1.000000,0.0,0.0,0.098976,0.000000,0.0,0.0,0.0,0.000000,0.000000,...,0.000000,0.000000,0.0,0.178006,0.000000,0.0,0.0,0.000000,0.0,0.000000
A022899328A0QROR32DCT,0.000000,1.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000
A100VQNP6I54HS,0.000000,0.0,1.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.052705
A100WO06OQR8BQ,0.098976,0.0,0.0,1.000000,0.009209,0.0,0.0,0.0,0.019932,0.057861,...,0.024451,0.000000,0.0,0.054335,0.000000,0.0,0.0,0.039409,0.0,0.000000
A100ZQDV7L8PVV,0.000000,0.0,0.0,0.009209,1.000000,0.0,0.0,0.0,0.011392,0.000000,...,0.000000,0.144013,0.0,0.000000,0.010545,0.0,0.0,0.000000,0.0,0.000000


In [48]:
df_similarity_user.loc['A00177463W0XWB16A9O05'].sort_values(ascending=False)[1:11].index

Index(['A266BFIUYON37V', 'AB29IDVYAYDL', 'A3ERTNN6GR1OU0', 'A2NGCLVURQIXXF',
       'A26Y776G7VVBCQ', 'A1UBSC80601DN1', 'A1NKR1WVFKDOT0', 'A1ZTR0FR5AW9B4',
       'AMZZ89NDUH0Q9', 'A1MDO8RZCZ40B0'],
      dtype='object', name='reviewerID')

In [49]:
idx_user = df_similarity_user.loc['A00177463W0XWB16A9O05'].sort_values(ascending=False)[1:11].index

In [50]:
serr = []
for user_id in idx_user:
    item_rating = user_based_pivot.loc[user_id]
    serr.append(item_rating)
    
(sum(serr)/len(idx_user)).sort_values(ascending=False).head(10)

title
Brooklyn Beans Breakfast Blend Decaffeinated CoffeeSingle-cup coffee for Keurig K-Cup Brewers for Keurig Brewers, 40 Count    3.7
Martinson Coffee Capsules, Dark Roast Package compatible with Keurig K-Cup Brewers, 48 Count                                  2.3
Keurig, The Original Donut Shop, 50 Count K-Cup Packs                                                                         1.9
Marley Coffee, Marley Mixer Single Serve RealCup Organic Variety Pack for Keurig K-Cup Brewers, 36 Count                      1.9
Brown Gold 100% Colombian Coffee Capsules, 48-Count Package compatible with Keurig K-Cup Brewers                              1.9
Grove Square Hot Cocoa, Dark, 24 Count Single Serve Cup for Keurig K-Cup Brewers                                              1.5
Cameron's Donut Shop Single Serve Coffees,  12-Count                                                                          1.3
Diedrich Coffee, Colombia, K-Cup Portion Pack for Keurig K-Cup Brewers (Pack of 50) 

In [51]:
import pickle

In [52]:
pickle.load(open('top25.sav', 'rb'))

,title,overall,count_review,score,imUrl,description,price
0,"Viva Labs Organic Extra Virgin Coconut Oil, 16...",4.790698,258,4.707921,http://ecx.images-amazon.com/images/I/51GJGJHv...,The #1 Coconut Oil for Freshness and PurityEve...,14.79
1,"Nature's Way Coconut Oil, 16 Ounce",4.775330,227,4.686826,http://ecx.images-amazon.com/images/I/51%2B66z...,Organic Coconut Oil by Nature's Way 16 oz Liqu...,12.57
2,"Nutiva Organic Extra Virgin Coconut Oil, 15-Ou...",4.773756,221,4.683697,http://ecx.images-amazon.com/images/I/51yPuYDg...,"Nutiva's cold-pressed, Organic extra-virgin co...",11.41
3,"Pamela's Ultimate Baking and Pancake Mix, 4-Po...",4.793103,145,4.663995,http://ecx.images-amazon.com/images/I/51d07Aqp...,"Create light and fluffy pancakes and waffles, ...",53.55
4,Y.S. Eco Bee Farms Raw Honey - 22 oz,4.819048,105,4.649206,http://ecx.images-amazon.com/images/I/4118A6wP...,Health Honey - Raw Unheated Unprocessed by Y.S...,6.39
5,The Spice Lab's Pure Himalayan Natural Unproce...,4.842697,89,4.645094,http://ecx.images-amazon.com/images/I/41bEdxA%...,The Spice Lab Finely Ground Pure Himalayan Salt,6.79
6,"Nutiva Organic Chia Seeds, 12-Ounce Bag",4.779528,127,4.642347,http://ecx.images-amazon.com/images/I/51OmOhsk...,"Chia seeds are a superfood packed with fiber, ...",3.12
7,U.S.A Grown Garbanzo Beans 5 LBS | 100% Non-Ir...,4.945455,55,4.635817,http://ecx.images-amazon.com/images/I/51IBb1DE...,Buy Garbanzo Beans grown from the fertile soil...,14.95
8,"Huy Fong, Sriracha Hot Chili Sauce, 17 Ounce B...",4.863014,73,4.632473,http://ecx.images-amazon.com/images/I/417IA%2B...,Sriracha is made from sun ripen chilies which ...,6.35
9,"Bob's Red Mill Gluten Free Whole Grain, Rolled...",4.853333,75,4.630511,http://ecx.images-amazon.com/images/I/51VXd3PG...,Bob's Red Mil Gluten Free Oats are pure. They...,34.00


In [ ]:

09